In [1]:
import json
import numpy as np
from scipy import stats

In [2]:
domain = "nlp"
cut_off = 500
pred_ft_file = "data/{}/outputs/bart-base-{}-{}-3epochs.lr1e-6.jsonl".format(domain, domain, cut_off)
pred_base_file = "data/{}/outputs/bart-base.jsonl".format(domain)
train_file = "data/{}/train.jsonl".format(domain)

In [3]:
# load prediction files

def load_pred(pred_file):
    predictions = {}
    with open(pred_file, 'r') as f:
        for line in f:
            doc = json.loads(line)
            predictions[doc["id"]] = doc["top_m"].split(";")
    return predictions

pred_ft = load_pred(pred_ft_file)
print(len(pred_ft))
pred_base = load_pred(pred_base_file)
print(len(pred_base))

212
212


In [4]:
# load training file
samples = {}
with open(train_file, 'r') as f:
    for line in f:
        if len(samples) == cut_off:
            break
        doc = json.loads(line)
        samples[doc["id"]] = doc["keyphrases"]
print(len(samples))     

500


In [5]:
# build a set of keyphrases from the training samples
sample_keyphrases = set()
for doc_id in samples:
    sample_keyphrases.update(samples[doc_id])
print(len(sample_keyphrases))

1297


In [6]:
# look for these keyphrases in the predicted phrases
m_ft = []
m_base = []
ratio_ft = []
ratio_base = []
for doc_id in pred_ft:
    m_ft.append(len(sample_keyphrases & set(pred_ft[doc_id])))
    m_base.append(len(sample_keyphrases & set(pred_base[doc_id])))
    ratio_ft.append(len(sample_keyphrases & set(pred_ft[doc_id]))/len(pred_ft[doc_id]))
    ratio_base.append(len(sample_keyphrases & set(pred_base[doc_id]))/len(pred_base[doc_id]))
    #print(doc_id, ratio)
print("base: {0:.2f} {1:.1f}".format(np.mean(m_base), np.mean(ratio_base)*100.0), "ft: {0:.2f} {1:.1f}".format(np.mean(m_ft), np.mean(ratio_ft)*100.0))
ttest = stats.ttest_rel(a=ratio_base, b=ratio_ft)
print('ratios: (sign@.05: {0}, pvalue: {1:.8f})'.format(ttest[1] < .05, ttest[1]))
ttest = stats.ttest_rel(a=m_base, b=m_ft)
print('counts: (sign@.05: {0}, pvalue: {1:.8f})'.format(ttest[1] < .05, ttest[1]))
print("difference: {0:.2f}".format( np.mean(m_ft) - np.mean(m_base)))

base: 0.87 20.9 ft: 0.83 20.8
ratios: (sign@.05: False, pvalue: 0.78821795)
counts: (sign@.05: False, pvalue: 0.05917488)
difference: -0.04
